In [14]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer
from tqdm import tqdm

### Carregar o dataset

In [15]:
df = pd.read_csv('../data/data_20230622/data.csv', sep=';')

In [16]:
df.head()

,ID,Label,UserId,UserName,UserRole,Request,SheetId,SheetName,Date,Parent1Name,Parent2Name,Parent3Name
0,0,47,9b705e6cd6,Luis,PARTICIPANT,5f6ae87ce45a8d000156814a,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
1,1,26,0f1a3956d7,Matilde,PARTICIPANT,5f6ae88be45a8d000156814c,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
2,2,40,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
3,3,48,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
4,4,41,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN


In [17]:
df = df[df['UserRole'] == 'PARTICIPANTBR']
df.head()

,ID,Label,UserId,UserName,UserRole,Request,SheetId,SheetName,Date,Parent1Name,Parent2Name,Parent3Name
14881,14881,Natureza,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14882,14882,Planeta,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14883,14883,Terra,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14884,14884,Flor,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14885,14885,O nosso planeta é muito bonito,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos


### Tratamento do dataset

In [18]:
# criar um novo dataframe com as colunas id e label
df = df[['ID', 'Label']]

In [19]:
# transformar a coluna id em string
df['ID'] = df['ID'].astype(str)

In [20]:
# adicionar uma string '../data/data_20230622/' no início da coluna id
df['ID'] = '../data/data_20230622/' + df['ID'] + '.jpeg'

In [21]:
# guardar esse novo dataframe em um arquivo csv
if not os.path.exists('labels_BR.csv'):    
    df.to_csv('labels.csv', index=False)
else :
    print('Arquivo já existe')    

Arquivo já existe


In [22]:
df.head()

,ID,Label
14881,../data/data_20230622/14881.jpeg,Natureza
14882,../data/data_20230622/14882.jpeg,Planeta
14883,../data/data_20230622/14883.jpeg,Terra
14884,../data/data_20230622/14884.jpeg,Flor
14885,../data/data_20230622/14885.jpeg,O nosso planeta é muito bonito


### Rodar o modelo

In [23]:
class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.vocab)[0]

        return text

In [24]:
model = ImageToWordModel("./Models/model.onnx")

In [25]:
def save_image(image, image_path, label, prediction_text, distance):
    
    plt.imshow(image)
    plt.title(f"Label: {label}, Prediction: {prediction_text}, Distance: {distance}")
    if not os.path.isfile(f"../data/imagens_BR/{image_path.split('/')[-1]}"): 
        plt.savefig(f"../data/imagens_BR/{image_path.split('/')[-1]}")
    else:
        print("existe")

In [28]:
accum_cer = []
for index, row in tqdm(df.iterrows()):
    image = cv2.imread(row['ID'])
    text = model.predict(image)
    label = row["Label"]
    result = get_cer(label.upper(), text.upper())
    if isinstance(result, float):
        cer = result
        distance = None  # Defina um valor padrão para a distância, se necessário
    else:
        cer, distance = result
    print(f'Label: {label} - Text: {text} - CER: {cer} - Distance: {distance}')
    accum_cer.append(cer)
    # save_image(image, row['ID'], label, text, distance)

print(f'Mean CER: {np.mean(accum_cer)}')


91it [00:00, 453.99it/s]

Label: Natureza - Text: NATUREZA - CER: 0.0 - Distance: 0
Label: Planeta - Text: PLANETA - CER: 0.0 - Distance: 0
Label: Terra - Text: TERRA - CER: 0.0 - Distance: 0
Label: Flor - Text: FLOR - CER: 0.0 - Distance: 0
Label: O nosso planeta é muito bonito - Text: O NOSSO PLANETA É MUITO BONITO - CER: 0.0 - Distance: 0
Label: Natureza - Text: NATURESA - CER: 0.125 - Distance: 1
Label: Planeta - Text: PLANETA - CER: 0.0 - Distance: 0
Label: Terra - Text: TERRA - CER: 0.0 - Distance: 0
Label: Flor - Text: FLOR - CER: 0.0 - Distance: 0
Label: Kauane Sofia Alves: O nosso planeta é muito bonito - Text: O NOSSO PLANETA É MUITO BONITO - CER: 0.6666666666666666 - Distance: 20
Label: NATUREVA - Text: NATUREZA - CER: 0.125 - Distance: 1
Label: PANETA - Text: PANETA - CER: 0.0 - Distance: 0
Label: TERA - Text: TERA - CER: 0.0 - Distance: 0
Label: FORE - Text: FLOR - CER: 0.5 - Distance: 2
Label: O NOSE PANETA E BONITO - Text: O NOSSO PLANETA É MUTO BONITO - CER: 0.3103448275862069 - Distance: 9
Labe

137it [00:00, 440.03it/s]

Label: TERRA - Text: TLRRA - CER: 0.2 - Distance: 1
Label: FO - Text: FO - CER: 0.0 - Distance: 0
Label: AUEA - Text: PLANETA - CER: 0.5714285714285714 - Distance: 4
Label: EDT - Text: FLR - CER: 1.0 - Distance: 3
Label: TA - Text: FA - CER: 0.5 - Distance: 1
Label: AI - Text: FO - CER: 1.0 - Distance: 2
Label: NATUREZA - Text: NATRETA - CER: 0.2857142857142857 - Distance: 2
Label: PLANETA - Text: TLTA - CER: 1.0 - Distance: 4
Label: TERRA - Text: FLR - CER: 1.3333333333333333 - Distance: 4
Label: FLOR - Text:  - CER: 0.0 - Distance: None
Label: NATUTEZA - Text: NATUREZA - CER: 0.125 - Distance: 1
Label: PATNETA - Text: PLANETA - CER: 0.2857142857142857 - Distance: 2
Label: LHETA - Text: PANETA - CER: 0.5 - Distance: 3
Label: FUTOU - Text: FLOR - CER: 0.75 - Distance: 3
Label: O NOSO PATNETA LHEMUH TO BONITO - Text: O NOSSO PLANETA É MUITO BONITO - CER: 0.26666666666666666 - Distance: 8
Label: NATURESA - Text: NATUREZA - CER: 0.125 - Distance: 1
Label: PLANETA - Text: NATURESA - CER: 0

226it [00:00, 426.79it/s]

...

Label: amor - Text: amor - CER: 0.0 - Distance: 0
Label: pais - Text: paO - CER: 0.6666666666666666 - Distance: 2
Label: eu gosto de reseber carinho - Text: eu gosto de receber carinho - CER: 0.037037037037037035 - Distance: 1
Label: felisidade - Text: feliidade - CER: 0.1111111111111111 - Distance: 1
Label: carinho - Text: carinho - CER: 0.0 - Distance: 0
Label: amor - Text: ALHO - CER: 0.75 - Distance: 3
Label: pas - Text: paz - CER: 0.3333333333333333 - Distance: 1
Label: eu gosto de receber carinho - Text: eu gosto de receber carinho - CER: 0.0 - Distance: 0
Label: frlisidade - Text: feliiade - CER: 0.375 - Distance: 3
Label: carinho - Text: carinho - CER: 0.0 - Distance: 0
Label: amor - Text: amor - CER: 0.0 - Distance: 0
Label: paz - Text: paO - CER: 0.3333333333333333 - Distance: 1
Label: eu gosto de  - Text: eu gosto de receber carinho - CER: 0.5555555555555556 - Distance: 15
Label: estomago - Text: feliade - CER: 1.0 - Distance: 7
Label: cabeça - Text: PaMETA - CER: 0.5 - Dist

11299it [00:27, 438.86it/s]

Label: estousalu - Text: CEBOLIHA - CER: 1.0 - Distance: 8
Label: cabsa - Text: cabsa - CER: 0.0 - Distance: 0
Label: udalp - Text: ap - CER: 1.5 - Distance: 3
Label: ancacirauqtn - Text: OOA - CER: 3.6666666666666665 - Distance: 11
Label: rhao - Text: amor - CER: 1.0 - Distance: 4
Label: aio - Text: ALO - CER: 0.3333333333333333 - Distance: 1
Label: ao - Text: AO - CER: 0.0 - Distance: 0
Label: pai - Text: pas - CER: 0.3333333333333333 - Distance: 1
Label: eu oo daio - Text: e - CER: 9.0 - Distance: 9
Label: etao - Text: eaA - CER: 0.6666666666666666 - Distance: 2
Label: kbc - Text: ALO - CER: 1.0 - Distance: 3
Label: pa - Text: pe - CER: 0.5 - Distance: 1
Label: e - Text: e - CER: 0.0 - Distance: 0
Label: ekbcipe v o iop - Text: e - CER: 14.0 - Distance: 14
Label: estomagu - Text: eBETA - CER: 1.2 - Distance: 6
Label: cabeça - Text: CabIsA - CER: 0.3333333333333333 - Distance: 2
Label: pena - Text: ALHO - CER: 1.0 - Distance: 4
Label: pe - Text: AO - CER: 1.0 - Distance: 2
Label: na-

11372it [00:27, 417.56it/s]

Label: felisidade - Text: feliidade - CER: 0.1111111111111111 - Distance: 1
Label: carinho - Text: CEBOLINHA - CER: 0.5555555555555556 - Distance: 5
Label: amor - Text: amor - CER: 0.0 - Distance: 0
Label: pas - Text: paz - CER: 0.3333333333333333 - Distance: 1
Label: eu gosto de receber carinho - Text: eu gosto de receber carinho - CER: 0.0 - Distance: 0
Label: felisidade - Text: feliidade - CER: 0.1111111111111111 - Distance: 1
Label: carino - Text: carinho - CER: 0.14285714285714285 - Distance: 1
Label: amor - Text: amor - CER: 0.0 - Distance: 0
Label: pasi - Text: AmO - CER: 1.0 - Distance: 3
Label: e goto de carino - Text: eu goto de Iebe caLiADA - CER: 0.43478260869565216 - Distance: 10
Label: felizidade - Text: feliidade - CER: 0.1111111111111111 - Distance: 1
Label: cario - Text: CAO - CER: 0.6666666666666666 - Distance: 2
Label: amor - Text: AO - CER: 1.0 - Distance: 2
Label: paz - Text: paO - CER: 0.3333333333333333 - Distance: 1
Label: eu gosto de cario - Text: eIca - CER: 3